# Imports

In [1]:
import pandas as pd
import re
from unidecode import unidecode

In [2]:
# Setar vars gerais
first_programa = 385
last_programa = 405

# Tomar nome de todos os países reconhecidos pela ONU

In [3]:
file_path = "nomes_paises_ptbr.txt"
with open(file_path, 'r', encoding='utf-8') as f:
    nomes_paises = f.read().splitlines()
print(len(nomes_paises),'paises no total.')
print(nomes_paises[0],'...',nomes_paises[-1])

201 paises no total.
Afeganistão ... Liechtenstein


# Tomar DataFrame Geral

In [4]:
df = pd.DataFrame(columns=['Pais', 'NumCit', 'Programa'])
for prognum in range(first_programa, last_programa+1):
    filename = f'TextFiles/xadrez_verbal_{prognum}.txt'
    with open(filename, 'r', encoding='utf-8') as f:
        texto = f.read()

    # Cria um dicionário para contar ocorrências
    contagem = {}

    # Normaliza o texto para fazer a busca sem diferenciar maiúsculas/minúsculas
    texto_normalizado = unidecode(texto.lower())

    # Para cada país, procura no texto
    for pais in nomes_paises:
        pais_normalizado = unidecode(pais.lower())
        padrao = r'\b' + re.escape(pais_normalizado) + r'\b'    # procura a palavra exata
        ocorrencias = len(re.findall(padrao, texto_normalizado))
        if ocorrencias > 0:
            contagem[pais] = ocorrencias

    # Cria o DataFrame com os resultados
    dfprog = pd.DataFrame(list(contagem.items()), columns=['Pais', 'NumCit'])

    # Ordena por número de citações (opcional)
    dfprog = dfprog.sort_values(by='NumCit', ascending=False).reset_index(drop=True)

    # Add número do programa 
    dfprog['Programa']=prognum

    # Add ao DataFrame principal
    df = pd.concat([df, dfprog])

df.sample(10)

,Pais,NumCit,Programa
27,França,4,394
47,Níger,2,401
32,Tonga,3,393
10,Ucrânia,8,401
27,Somália,3,399
38,Etiópia,1,396
59,Líbano,1,392
32,Suécia,5,403
50,Iraque,1,400
12,Líbano,11,400


In [5]:
df.to_csv('Data/Geral.csv', header=True, index=False)

# Groupby de citações gerais por países e por programa

In [6]:
df_numcitacoes = df.groupby(by='Pais')['NumCit'].sum().reset_index().sort_values(by='NumCit', ascending=False)
df_numcitacoes

,Pais,NumCit
43,Estados Unidos,1315
15,Brasil,942
69,Israel,835
28,China,453
120,Rússia,432
...,...,...
96,Montenegro,1
29,Chipre,1
124,Serra Leoa,1
115,Quênia,1


In [7]:
df_numcitacoes.to_csv('Data/Numero_Citacoes.csv', header=True, index=False)

In [8]:
df_numprogramas = df.groupby(by='Pais')['Programa'].count().reset_index().rename(columns={'Programa': 'NumProgramas'}).sort_values(by='NumProgramas', ascending=False)
df_numprogramas

,Pais,NumProgramas
4,Argentina,21
2,Alemanha,21
14,Bolívia,21
15,Brasil,21
69,Israel,21
...,...,...
94,Micronésia,1
124,Serra Leoa,1
140,Turcomenistão,1
138,Trinidad e Tobago,1


In [9]:
df_numprogramas.to_csv('Data/Numero_Programas.csv', header=True, index=False)

# Relação de países mais relacionados

In [10]:
# Função para identificar dois países juntos utilizando regex
def encontrar_paises_juntos(texto, nomes_paises):
    texto_normalizado = unidecode(texto.lower())  # Normaliza o texto para uma comparação sem acento
    contagem = {}

    # Para cada país na lista, procurar no texto
    for pais in nomes_paises:
        pais_normalizado = unidecode(pais.lower())  # Normaliza o nome do país para comparação sem acento
        padrao = r'\b' + re.escape(pais_normalizado) + r'\b'  # Cria o padrão para a palavra exata
        ocorrencias = len(re.findall(padrao, texto_normalizado))  # Conta quantas vezes o país aparece
        if ocorrencias > 0:
            contagem[pais] = ocorrencias  # Se o país aparece, registra a contagem

    # Verificar pares de países juntos
    pares = []
    for pais1 in contagem:
        for pais2 in contagem:
            if pais1 != pais2 and pais1 < pais2:  # Não verificar o mesmo país e evita duplicação (Sudão-Índia e Índia-Sudão)
                if pais1 in contagem and pais2 in contagem:
                    pares.append((pais1, pais2))

    return pares

## Tabela com minutagem de cada citação

In [11]:
df_pares = pd.DataFrame(columns=['programa', 'start_segment', 'end_segment', 'pais1', 'pais2'])
for prognum in range(first_programa, last_programa+1):
    print('# Iniciando programa ',prognum)
    # Carregar tabela daquele programa
    dft = pd.read_csv(f'Tables/xadrez_verbal_{prognum}.csv')

    ocorrencias_duplas = []
    for index, row in dft.iterrows():
        texto = row['text']
        if type(texto)==str:
            pares_encontrados = encontrar_paises_juntos(texto, nomes_paises)
            
            for par in pares_encontrados:
                ocorrencias_duplas.append({
                    "programa": prognum,
                    "start_segment": row['start_segment'],
                    "end_segment": row['end_segment'],
                    "pais1": par[0],
                    "pais2": par[1]
                })

    df_pares = pd.concat([df_pares, pd.DataFrame(ocorrencias_duplas)])
print('# Concluído!')

# Iniciando programa  385


C:\Users\leona\AppData\Local\Temp\ipykernel_9732\2428359073.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_pares = pd.concat([df_pares, pd.DataFrame(ocorrencias_duplas)])


# Iniciando programa  386
# Iniciando programa  387
# Iniciando programa  388
# Iniciando programa  389
# Iniciando programa  390
# Iniciando programa  391
# Iniciando programa  392
# Iniciando programa  393
# Iniciando programa  394
# Iniciando programa  395
# Iniciando programa  396
# Iniciando programa  397
# Iniciando programa  398
# Iniciando programa  399
# Iniciando programa  400
# Iniciando programa  401
# Iniciando programa  402
# Iniciando programa  403
# Iniciando programa  404
# Iniciando programa  405
# Concluído!


In [12]:
def ordenar_paises(par):
    pais1, pais2 = sorted(par)
    return (pais1, pais2)

df_pares['pais_par'] = df_pares[['pais1', 'pais2']].apply(ordenar_paises, axis=1)

df_pares

,programa,start_segment,end_segment,pais1,pais2,pais_par
0,385,826.00,835.00,Etiópia,Somália,"(Etiópia, Somália)"
1,385,863.00,867.00,Etiópia,Somália,"(Etiópia, Somália)"
2,385,945.00,951.00,Etiópia,Turquia,"(Etiópia, Turquia)"
3,385,951.00,957.00,Etiópia,Somália,"(Etiópia, Somália)"
4,385,963.00,972.00,Etiópia,Somália,"(Etiópia, Somália)"
...,...,...,...,...,...,...
113,405,20514.52,20525.52,China,Estados Unidos,"(China, Estados Unidos)"
114,405,20583.52,20596.52,Coréia do Norte,Venezuela,"(Coréia do Norte, Venezuela)"
115,405,21089.52,21094.52,Angola,Estados Unidos,"(Angola, Estados Unidos)"
116,405,21161.52,21166.52,Angola,Brasil,"(Angola, Brasil)"


In [13]:
df_pares.to_csv('Data/Relacoes_Paises_Minutagem.csv', index=False, header=True)

## Paises mais relacionados entre si


In [14]:
result = df_pares.groupby('pais_par').size().reset_index(name='num_relac_paises')
result[['pais1', 'pais2']] = pd.DataFrame(result['pais_par'].tolist(), index=result.index)
result.sort_values(by='num_relac_paises', ascending=False)

,pais_par,num_relac_paises,pais1,pais2
338,"(Estados Unidos, Israel)",43,Estados Unidos,Israel
238,"(China, Rússia)",34,China,Rússia
221,"(China, Estados Unidos)",33,China,Estados Unidos
247,"(China, Índia)",30,China,Índia
41,"(Argentina, Brasil)",30,Argentina,Brasil
...,...,...,...,...
581,"(Ucrânia, Venezuela)",1,Ucrânia,Venezuela
195,"(Canadá, Finlândia)",1,Canadá,Finlândia
583,"(Zâmbia, África do Sul)",1,Zâmbia,África do Sul
584,"(África do Sul, Índia)",1,África do Sul,Índia


In [15]:
result.to_csv('Data/Relacoes_Paises_Contagens.csv', index=False, header=True)